# 01_임베딩 3D 시각화

# 0. 기본 Library

In [ ]:
import pandas as pd
import numpy as np

## 1. OpenAI 라이브러리 설치 확인

In [ ]:
# !pip install openai python-dotenv ipympl

## 2. OpenAI 정보 읽기

In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
dotenv_path = find_dotenv(filename='./.env')
load_dotenv(dotenv_path=dotenv_path)

openai.api_type = "azure"
openai.api_version = os.getenv("OPENAI_API_VERSION","").strip()

API_KEY = os.getenv("OPENAI_API_KEY","").strip()
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

deployment_id ='text-embedding-ada-002'

## 3. 함수 정의

In [ ]:
def get_embedding(text, deployment_id=deployment_id):
    """ 
    Get embeddings for an input text from the dataframe. 
    """
    result = openai.Embedding.create(
      deployment_id=deployment_id,
      input=text
    )
    result = np.array(result["data"][0]["embedding"])
    return result

def vector_similarity(x, y):
    """
    Returns the similarity between two vectors.    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    similarity = np.dot(x, y)
    return similarity 

def order_document_sections_by_query_similarity(query, contexts):
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated articles embeddings
    to find the most relevant articles. 
    Return the list of articles, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)

    document_similarities = sorted(
        [(vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()], 
        reverse=True)
    
    return document_similarities

## 4. 데이터 소스 Embedding

In [ ]:
df_orig = pd.read_csv("../data/bbc-news-data.csv", delimiter='\t')
df = df_orig.copy()
df

,category,filename,title,content
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...
...,...,...,...,...
2220,tech,397.txt,BT program to beat dialler scams,BT is introducing two initiatives to help bea...
2221,tech,398.txt,Spam e-mails tempt net shoppers,Computer users across the world continue to i...
2222,tech,399.txt,Be careful how you code,A new European directive could put software w...
2223,tech,400.txt,US cyber security chief resigns,The man making sure US computer networks are ...


In [ ]:
df['embedding'] = ''

# for i in range(len(df)):    
for i in range(505,520):
    try:
        embedding = openai.Embedding.create(input=df['content'][i], deployment_id=deployment_id)
        df['embedding'][i] = np.array(embedding['data'][0]['embedding'])
    except Exception as err:
        i
        print(f"Unexpected {err=}, {type(err)=}")

df = df[df['embedding'].apply(lambda x: isinstance(x, (list, np.ndarray)))]

df

,category,filename,title,content,embedding
505,business,506.txt,Trial begins of Spain's top banker,"The trial of Emilio Botin, the chairman of Sp...","[-0.0185097549110651, -0.017938129603862762, 0..."
506,business,507.txt,UK economy ends year with spurt,The UK economy grew by an estimated 3.1% in 2...,"[-0.02052326686680317, -0.023954439908266068, ..."
507,business,508.txt,HealthSouth ex-boss goes on trial,The former head of US medical services firm H...,"[-0.006607617251574993, 0.0022741160355508327,..."
508,business,509.txt,Euro firms miss out on optimism,More than 90% of large companies around the w...,"[0.0009200896602123976, -0.03431011736392975, ..."
509,business,510.txt,Lacroix label bought by US firm,Luxury goods group LVMH has sold its loss-mak...,"[-0.018210988491773605, 0.007721032947301865, ..."
510,entertainment,001.txt,Gallery unveils interactive tree,A Christmas tree that can receive text messag...,"[-0.031129341572523117, -0.015320412814617157,..."
511,entertainment,002.txt,Jarre joins fairytale celebration,French musician Jean-Michel Jarre is to perfo...,"[-0.004232572857290506, -0.02793629840016365, ..."
512,entertainment,003.txt,Musical treatment for Capra film,The classic film It's A Wonderful Life is to ...,"[0.004460970871150494, -0.03234203904867172, -..."
513,entertainment,004.txt,Richard and Judy choose top books,The 10 authors shortlisted for a Richard and ...,"[-0.02381858415901661, -0.016464833170175552, ..."
514,entertainment,005.txt,Poppins musical gets flying start,The stage adaptation of children's film Mary ...,"[0.00714197289198637, -0.033781010657548904, -..."


## 5. 질의 기반 검색

- 문서를 사전에 임베딩 시켜놓고 나중에 질의
- 코사인 각도로 질의와 답변 간 임베딩 거리 상 유사성 확인 가능

In [ ]:
query = 'News about stock market.'

In [ ]:
answers = order_document_sections_by_query_similarity(query=query, contexts=df['embedding'])[0:3] # Set to top 3

print(answers)

# print top 3
for answer in answers:
    print(f'similarity score:   {answer[0]}')
    print(df['content'].loc[answer[1]], '\n')

[(0.7809996954868466, 508), (0.7525378803451948, 506), (0.7508318237574978, 507)]
similarity score:   0.7809996954868466
 More than 90% of large companies around the world are highly optimistic about their economic prospects, a survey of 1,300 bosses suggests.  Their biggest worries are not terror threats, but over-regulation, low-cost competition and the wild ups and downs of oil prices. There is one exception: Firms in Western Europe - but not the UK - are lacking confidence after years of slow growth. When business advisers PricewaterhouseCoopers (PwC) conducted the same survey two years ago, nearly 30% of bosses were gloomy about their prospects.  Global business leaders say that they are facing a two-pronged regulatory assault. After a string of corporate scandals in the United States - from Enron to WorldCom - the Sarbanes-Oxley act forces companies to be much more transparent, but doing all the paperwork costs a lot of time and money. Across Europe, meanwhile, all stock exchange

## 6. Embedding 시각화

### Question 시각화 데이터 추가 및 검색 결과 데이터 변경

In [9]:
for answer in answers:
    df.at[answer[1], 'category'] = df.loc[answer[1], 'category'] + "/answer"


embedding = openai.Embedding.create(input=query, deployment_id=deployment_id)

new_row = {'category': 'Question', 'content': query, 'embedding': np.array(embedding['data'][0]['embedding'])}
    
new_df = pd.DataFrame([new_row])

df = pd.concat([df, new_df], ignore_index=True)

### 차원 줄이기

In [10]:
if False: # Set to True if using TSNE
    from sklearn.manifold import TSNE

    # Create a t-SNE model and transform the data
    tsne = TSNE(n_components=3, perplexity=15, random_state=42, init='random', learning_rate=200)
    vis_dims_tsne = tsne.fit_transform(df['embedding'].to_list())
    vis_dims_tsne.shape

In [11]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
vis_dims_pca = pca.fit_transform(df['embedding'].to_list())
vis_dims_pca.shape
vis_dims_pca

array([[ 0.21497269, -0.05565156,  0.17946016],
       [ 0.23318004,  0.24007962, -0.14144674],
       [ 0.24933358,  0.0148129 ,  0.25016281],
       [ 0.27870648,  0.09678162,  0.0082348 ],
       [ 0.1472603 , -0.17498888, -0.06434964],
       [-0.05593049, -0.1745074 , -0.23564962],
       [-0.09109641, -0.18961528, -0.11713396],
       [-0.13628988, -0.126475  ,  0.09113007],
       [-0.14870753,  0.23673202, -0.15031481],
       [-0.138569  , -0.12466213,  0.06420554],
       [-0.24968454,  0.17350542,  0.15132495],
       [-0.17204941,  0.19437548, -0.13722851],
       [-0.26940748,  0.12767939,  0.18321807],
       [-0.02376559, -0.11891207, -0.14633375],
       [-0.11855073, -0.16663561,  0.10968414],
       [ 0.28059797,  0.04748146, -0.04496352]])

In [12]:
df["vis_dims_pca"] = vis_dims_pca.tolist()

### 차트 시각화

In [ ]:
%matplotlib widget 
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(projection='3d')
cmap = plt.get_cmap("tab20")

categories = sorted(df['category'].unique())

# Plot each sample category individually
for i, cat in enumerate(categories):
    sub_matrix = np.array(df[df["category"] == cat]["vis_dims_pca"].to_list())
    x=sub_matrix[:, 0]
    y=sub_matrix[:, 1]
    z=sub_matrix[:, 2]
    colors = [cmap(i/len(categories))] * len(sub_matrix)
    _ = ax.scatter(x, y, zs=z, zdir='z', c=colors, label=cat)

_ = ax.set_xlabel('x')
_ = ax.set_ylabel('y')
_ = ax.set_zlabel('z')
_ = ax.legend()